Apprentissage de données $\phi$-FEM

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

seed = 2023
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.experimental.numpy.random.seed(seed)
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
os.environ["TF_DETERMINISTIC_OPS"] = "1"
# Set a fixed value for the hash seed
os.environ["PYTHONHASHSEED"] = str(seed)
import dolfin as df
import time
from utils import *
from utils_training import *

import seaborn as sns

sns.set_theme()
sns.set_context("paper")
sns.set(rc={"xtick.bottom": True, "ytick.left": True})
colors = sns.color_palette("mako").as_hex()
my_cmap = sns.color_palette("viridis", as_cmap=True)

# Run

In [ ]:
nb_layers = [4, 5, 6]
width = [10, 20, 50, 100]
modes = [5, 10, 20, 25]
levels = [0, 1, 2]

params = np.array(np.meshgrid(nb_layers, width, modes)).T.reshape(-1, 3)
new_params = np.zeros((np.shape(params)[0] * 3, 4))
for i in range(3):
    new_params[
        np.shape(params)[0] * (i) : np.shape(params)[0] * (i + 1), :3
    ] = params[:, :]
    new_params[
        np.shape(params)[0] * (i) : np.shape(params)[0] * (i + 1), 3
    ] = i
np.save("./params.npy", new_params)
params = np.load("./params.npy")
print(np.shape(params))

In [ ]:
small_data = False
data = DataLoader(small_data)


In [ ]:
index = 0
for param in new_params[index:]:
    print(f"{index}/{len(new_params)}")
    nb_layer, width_i, modes_i, level = param
    nb_layer, width_i, modes_i, level = (
        int(nb_layer),
        int(width_i),
        int(modes_i),
        int(level),
    )
    agent = Agent(
        data,
        small_data,
        width=width_i,
        modes=modes_i,
        nb_layers=nb_layer,
        index_test_case=index,
    )
    epochs = 202
    ti0 = time.time()
    agent.train(level, epochs)
    duration = time.time() - ti0
    print("average time per epoch :", duration / epochs)
    index += 1